<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 3 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [0]:
!curl https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.names

In [22]:
import pandas as pd
columns=['Party', 'handicapped_infants', 'water_project','adoption_budget','physician_fee','salvador_aid', 'religious_groups','satellite_test','aid_to_nicaraguan', 'mx_missile',
       'immigration', 'synfuels_cutback','education_spending','superfund_sue','crime','exports','export_south-africa']

df=pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data', names= columns, na_values='?')
print(df.shape)
df.head()

(435, 17)


,Party,handicapped_infants,water_project,adoption_budget,physician_fee,salvador_aid,religious_groups,satellite_test,aid_to_nicaraguan,mx_missile,immigration,synfuels_cutback,education_spending,superfund_sue,crime,exports,export_south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,NaN,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,NaN
2,democrat,NaN,y,y,NaN,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,NaN,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,NaN,y,y,y,y


In [23]:
df=df.replace(['y','n'],[1,0])
print(df.shape)
df.head()

(435, 17)


,Party,handicapped_infants,water_project,adoption_budget,physician_fee,salvador_aid,religious_groups,satellite_test,aid_to_nicaraguan,mx_missile,immigration,synfuels_cutback,education_spending,superfund_sue,crime,exports,export_south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [24]:
df.isna().sum()

Party                    0
handicapped_infants     12
water_project           48
adoption_budget         11
physician_fee           11
salvador_aid            15
religious_groups        11
satellite_test          14
aid_to_nicaraguan       15
mx_missile              22
immigration              7
synfuels_cutback        21
education_spending      31
superfund_sue           25
crime                   17
exports                 28
export_south-africa    104
dtype: int64

In [25]:
df.describe()

,handicapped_infants,water_project,adoption_budget,physician_fee,salvador_aid,religious_groups,satellite_test,aid_to_nicaraguan,mx_missile,immigration,synfuels_cutback,education_spending,superfund_sue,crime,exports,export_south-africa
count,423.000000,387.000000,424.000000,424.000000,420.000000,424.000000,421.000000,420.00000,413.000000,428.000000,414.000000,404.000000,410.000000,418.000000,407.000000,331.000000
mean,0.442080,0.503876,0.596698,0.417453,0.504762,0.641509,0.567696,0.57619,0.501211,0.504673,0.362319,0.423267,0.509756,0.593301,0.427518,0.812689
std,0.497222,0.500632,0.491140,0.493721,0.500574,0.480124,0.495985,0.49475,0.500605,0.500563,0.481252,0.494690,0.500516,0.491806,0.495327,0.390752
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000
75%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [0]:
df=df.fillna(df.mean())

In [27]:
df.isna().sum()

Party                  0
handicapped_infants    0
water_project          0
adoption_budget        0
physician_fee          0
salvador_aid           0
religious_groups       0
satellite_test         0
aid_to_nicaraguan      0
mx_missile             0
immigration            0
synfuels_cutback       0
education_spending     0
superfund_sue          0
crime                  0
exports                0
export_south-africa    0
dtype: int64

In [28]:
democrat= df[df['Party']=='democrat']
print(democrat.shape)
democrat.head()

(267, 17)


,Party,handicapped_infants,water_project,adoption_budget,physician_fee,salvador_aid,religious_groups,satellite_test,aid_to_nicaraguan,mx_missile,immigration,synfuels_cutback,education_spending,superfund_sue,crime,exports,export_south-africa
2,democrat,0.44208,1.0,1.0,0.417453,1.000000,1.0,0.0,0.0,0.0,0.0,1.0,0.000000,1.000000,1.0,0.0,0.0
3,democrat,0.00000,1.0,1.0,0.000000,0.504762,1.0,0.0,0.0,0.0,0.0,1.0,0.000000,1.000000,0.0,0.0,1.0
4,democrat,1.00000,1.0,1.0,0.000000,1.000000,1.0,0.0,0.0,0.0,0.0,1.0,0.423267,1.000000,1.0,1.0,1.0
5,democrat,0.00000,1.0,1.0,0.000000,1.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.000000,1.0,1.0,1.0
6,democrat,0.00000,1.0,0.0,1.000000,1.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.509756,1.0,1.0,1.0


In [29]:
repub= df[df['Party']=='republican']
print(repub.shape)
repub.head()

(168, 17)


,Party,handicapped_infants,water_project,adoption_budget,physician_fee,salvador_aid,religious_groups,satellite_test,aid_to_nicaraguan,mx_missile,immigration,synfuels_cutback,education_spending,superfund_sue,crime,exports,export_south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.362319,1.000000,1.0,1.0,0.000000,1.000000
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.000000,1.000000,1.0,1.0,0.000000,0.812689
7,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,1.0,1.0,0.427518,1.000000
8,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.000000,1.000000,1.0,1.0,0.000000,1.000000
10,republican,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.362319,0.423267,1.0,1.0,0.000000,0.000000


In [38]:
print('democrat support: ', democrat['handicapped_infants'].mean())
print('republican support: ',repub['handicapped_infants'].mean())

democrat support:  0.5991712487050762
republican support:  0.19241810199257006


In [31]:
# calculation of tstat, pvalue
from scipy.stats import ttest_ind, ttest_ind_from_stats, ttest_rel
import numpy as np
tstat,pvalue= ttest_ind(democrat['handicapped_infants'], repub['handicapped_infants'])
print(tstat)
print(pvalue)

9.200405974189513
1.5238337420363941e-18


Null hypothesis - No difference between parties, they supported the bill equally

Null hypothesis- the mean of democrat support== the mean of the republican support

Alternative Hypothesis- the means are different(not the same level of support)

T-statistic goes up and pvalue (p<0.01) goes down
Given the result of the above test i would reject the null hypothesis that there is same level of support for this bill among democrats and republican


In [42]:
# second part of the assignment
for feat in df.columns:
  group1= df[df['Party']=='democrat'][feat]
  group2= df[df['Party']=='republican'][feat]
  # run tstat, pvalue
  tstat,pvalue= ttest_ind(group1,group2)

TypeError: ignored

Hypothesis - 

In [34]:
#percentage of democrat supported the bill
democrat['handicapped_infants'].mean()

0.5991712487050762

In [33]:
democrat['handicapped_infants'].value_counts()
# 9 people abstrain from voting

1.00000    156
0.00000    102
0.44208      9
Name: handicapped_infants, dtype: int64

In [0]:
#df[df['y.8'].isna()]